In [ ]:
! pip install langchain
! pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.1,
                 model=llm_model,
                 openai_api_key= "YourChatGPTAPIKey")

In [ ]:
ADHD_Inattention = """You are a pediatric expert, and use the following suggestions if the question is related to ADHD Inattention.

Suggestions:
Provide daily structure and stick with it.\
Minimize distractions when doing tasks such as homework; child should not be in a high traffic level in the house , by the TV, or  having siblings playing close by.\
Break down tasks into simple sequential steps.\
Consider using timers, clocks as visual cues for the child.\
Allow child to stand during tasks like homework.\
Give breaks every 15 minutes.\
Use a daily planner.\
Reinforce good behavior with treats, tokens, tickets.\
Organize your house.\
Call child's name before giving directions; make sure he/she is focused on what you are saying.\
Establish good eye contact as a way to make sure he's attending.\
Have all materials in one place -e.\g.\, backpack.\
Focus on child's strengths.\


Here is a question:
{input}"""


ADHD_HI = """You are a pediatric expert, and use the following suggestions if the question is related to ADHD Hyperactive or Impulsive behavior.\

Suggestions:
If you want a behavior to continue, reward it.\
Charts should be used to remind child of chores.\
Schedule should be simplified.\
Verbal message should be in the first 2 sentences, otherwise impulsivity will get in the way.\
Limit screen time (can be used as a reward).\
Have child engage in sports.\
Child should engage in physical activity daily.\
Pick your battles.\
Don't sweat the small stuff.\
Provide clear expectations and rules.\
Be consistent.\
Rules are inflexible.\
Use charts.\
Reward appropriate behavior, rewards include access to a grab-bag, tokens, tickets.\
Use applicable items from the inattention section.\
Limit choices (consider only several options at a time.\
If you do not like a behavior but it's not dangerous or excessively disruptive, ignore it.\
If the behavior is dangerous or aggressive, punish it or have time out.\
Behaviors should change in small increments.\



Here is a question:
{input}"""


# history_template = """You are a very good historian. \
# You have an excellent knowledge of and understanding of people,\
# events and contexts from a range of historical periods. \
# You have the ability to think, reflect, debate, discuss and \
# evaluate the past. You have a respect for historical evidence\
# and the ability to make use of it to support your explanations \
# and judgements.

# Here is a question:
# {input}"""


# computerscience_template = """ You are a successful computer scientist.\
# You have a passion for creativity, collaboration,\
# forward-thinking, confidence, strong problem-solving capabilities,\
# understanding of theories and algorithms, and excellent communication \
# skills. You are great at answering coding questions. \
# You are so good because you know how to solve a problem by \
# describing the solution in imperative steps \
# that a machine can easily interpret and you know how to \
# choose a solution that has a good balance between \
# time complexity and space complexity.

# Here is a question:
# {input}"""

In [ ]:
prompt_infos = [
    {
        "name": "ADHD Inattention",
        "description": "Good for giving suggestions about ADHD Inattention",
        "prompt_template": ADHD_Inattention
    },
    {
        "name": "ADHD H/I",
        "description": "Good for giving suggestions about ADHD Hyperactive/Impulsive problems",
        "prompt_template": ADHD_HI
    },
    # {
    #     "name": "History",
    #     "description": "Good for answering history questions",
    #     "prompt_template": history_template
    # },
    # {
    #     "name": "computer science",
    #     "description": "Good for answering computer science questions",
    #     "prompt_template": computerscience_template
    # }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate


In [ ]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
final1 = chain.run("During reading time, Sarah often stares out the window, completely missing the teacher's questions about the story.As a parent, what should I do?")
print(final1)



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


ADHD Inattention: {'input': "During reading time, Sarah often stares out the window, completely missing the teacher's questions about the story.As a parent, what should I do?"}
> Finished chain.
As a parent, there are several strategies you can try to help Sarah improve her focus during reading time:

1. Provide daily structure and stick with it: Establish a consistent routine for reading time and ensure that it is a quiet and distraction-free environment.

2. Minimize distractions: Make sure Sarah is not sitting near a window or any other potential distractions. Remove any toys or electronic devices that may divert her attention.

3. Break down tasks into simple sequential steps: Help Sarah understand the expectations during reading time by breaking down the activity into smaller, manageable steps. For example, you can ask her to read a paragraph and then summarize what she read.

4. Use visual cues: Consider using a timer or clock to visually indicate the duration of reading time. Th

In [ ]:
final2 = chain.run("Tommy frequently interrupts the teacher during class discussions, blurting out answers without raising his hand. As a parent, what should I do?")
print(final2)



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


ADHD H/I: {'input': 'Tommy frequently interrupts the teacher during class discussions, blurting out answers without raising his hand. As a parent, what should I do?'}
> Finished chain.
As a parent, there are several strategies you can try to address Tommy's impulsive behavior in the classroom:

1. Provide clear expectations and rules: Talk to Tommy about the importance of raising his hand and waiting for his turn to speak during class discussions. Explain why it is important to respect the teacher and other students.

2. Use charts: Create a visual chart or reminder for Tommy that highlights the expected behavior in the classroom. This can serve as a visual cue to remind him to raise his hand before speaking.

3. Reward appropriate behavior: When Tommy successfully raises his hand and waits for his turn to speak, provide positive reinforcement. This can be in the form of verbal praise, a small reward, or access to a preferred activity.

4. Limit screen time: Consider using screen time 